In [14]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
integers = tokenizer.encode("Akwirw ier")
print("Encoded: ", integers)

for i in integers:
    print(f"{i}\t=> {tokenizer.decode([i])}")

torch version: 2.5.1
tiktoken version: 0.8.0
Encoded:  [33901, 86, 343, 86, 220, 959]
33901	=> Ak
86	=> w
343	=> ir
86	=> w
220	=>  
959	=> ier


In [15]:
tokenizer.encode("Ak")

[33901]

In [16]:
tokenizer.encode("w")

[86]

In [17]:
tokenizer.encode("ir")

[343]

In [18]:
tokenizer.encode(" ")

[220]

In [19]:
tokenizer.encode("ier")

[959]

In [20]:
tokenizer.decode([33901, 86, 343, 86, 220, 959])

'Akwirw ier'

# Solution for 2.2

In [23]:
import tiktoken
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# End Class GPTDatasetV1

def create_dataloader(txt, batch_size=4, max_length=256, stride=128):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")
    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

# read the book and tokenize
with open("../../data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    f.close()
tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)

vocab_size = 50257
output_dim = 256
max_lenth = 4
context_length = max_lenth
token_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


In [40]:
dataloader = create_dataloader(raw_text, batch_size=4, max_length=2, stride=2)
for batch in dataloader:
    x, y = batch
    break
print(f"X:\n{x}\nY:\n{y}")

X:
tensor([[  40,  367],
        [2885, 1464],
        [1807, 3619],
        [ 402,  271]])
Y:
tensor([[  367,  2885],
        [ 1464,  1807],
        [ 3619,   402],
        [  271, 10899]])


In [39]:
dataloader = create_dataloader(raw_text, batch_size=4, max_length=8, stride=2)
for batch in dataloader:
    x, y = batch
    break
print(f"X:\n{x}\nY:\n{y}")

X:
tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438]])
Y:
tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [  271, 10899,  2138,   257,  7026, 15632,   438,  2016]])
